In [14]:
"""
VerimGören — Noktadan (Google Maps linki **veya** lat,lon) → ürün uygunluk skoru (v0.4)

Kullanım akışı:
1) Program çalışınca konumu **Google Maps linki** YA DA **"lat,lon"** biçiminde girin.
   Örnekler:  
   - https://www.google.com/maps/@38.946838,28.080573,12z  
   - https://www.google.com/maps/place/41°52'10.0\"N+33°53'29.0\"E  
   - 38.946838, 28.080573
2) Betik Google Maps linkini yeniden yazdırır (kamera & sorgu linki), NASA POWER 1991–2020 aylık
   klimatolojiyi çeker (cache'li & retry'li), (opsiyonel) rasterlardan rakım/ışık örnekler,
   pilot ürünler için 0–100 uygunluk skorlarını ve gerekçelerini üretir.
3) Çıktıları ekrana yazar, ayrıca **results.json** ve **results.csv** dosyalarına kaydeder.

İyileştirmeler (v0.4):
- Giriş: Link **veya** lat,lon (akıllı ayrıştırma)
- Sulama modu anahtarı: rainfed / irrigated → su bileşeni ağırlığı dinamik
- Ürün profilleri **harici JSON** ile yüklenebilir (yoksa dahili varsayılanlar)
- NASA POWER çağrıları: retry + basit disk cache (./cache/power_climatology_cache.json)
- JSON/CSV çıktı dökümleri
"""

import os
import re
import csv
import json
import math
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import time
import requests

try:
    import rasterio  # opsiyonel
except Exception:  # rasterio yoksa sessiz devam
    rasterio = None

# -----------------------------
# 0) Kullanıcı değişkenleri
# -----------------------------

# Sulama modu: "rainfed" (kuru koşul) veya "irrigated" (sulamalı)
IRRIGATION_MODE = "rainfed"  # "rainfed" | "irrigated"

# Ürün profili harici konfigürasyon dosyası (opsiyonel)
CROP_CONFIG_PATH = Path("./crop_profiles.json")

# Opsiyonel raster yolları (varsa rapora eklenir)
ELEV_CROPPED_PATH = Path("../data/processed/srtm_turkiye_cropped.tif")
LIGHT_CROPPED_PATH = Path("../data/processed/viirs_light_2024_turkey.tif")

# Cache dosyası (NASA POWER klimatoloji)
CACHE_DIR = Path("./cache")
CACHE_DIR.mkdir(parents=True, exist_ok=True)
POWER_CACHE_FILE = CACHE_DIR / "power_climatology_cache.json"

# -----------------------------
# 1) Yardımcılar — Koordinat ayrıştırma & link üretimi
# -----------------------------

DMS_PATTERN = re.compile(r"(?P<deg>\d{1,3})°(?P<min>\d{1,2})'(?P<sec>[\d\.]+)\"(?P<hemi>[NSEW])")


def dms_to_decimal(deg: float, minute: float, sec: float, hemi: str) -> float:
    sign = -1 if hemi.upper() in ["S", "W"] else 1
    return sign * (abs(deg) + minute / 60.0 + sec / 3600.0)


def parse_latlon_text(text: str) -> Tuple[float, float]:
    """'lat,lon' düz metninden (lat, lon) döndürür; boşluk/virgül/; tolere eder."""
    text = text.strip()
    parts = re.split(r"[,\s;]+", text)
    parts = [p for p in parts if p]
    if len(parts) < 2:
        raise ValueError("Lütfen 'lat,lon' biçiminde iki sayı girin. Örn: 38.946838, 28.080573")
    lat = float(parts[0])
    lon = float(parts[1])
    if not (-90.0 <= lat <= 90.0 and -180.0 <= lon <= 180.0):
        raise ValueError("Geçersiz aralık: enlem [-90,90], boylam [-180,180] olmalı.")
    return lat, lon


def parse_google_maps_link(text: str) -> Optional[Tuple[float, float]]:
    """Google Maps linkinden (lat, lon) döndürür; '@lat,lon' ve DMS biçimlerini destekler."""
    text = text.strip()
    # 1) '@lat,lon' biçimi
    if "@" in text:
        try:
            after = text.split("@", 1)[1]
            nums = re.split(r"[^-\d\.]+", after)
            nums = [n for n in nums if n]
            lat = float(nums[0])
            lon = float(nums[1])
            return lat, lon
        except Exception:
            pass
    # 2) DMS kalıbı (iki adet: lat, lon)
    dms_hits = DMS_PATTERN.findall(text)
    if len(dms_hits) >= 2:
        (ld, lm, ls, lh), (od, om, os, oh) = dms_hits[0], dms_hits[1]
        lat = dms_to_decimal(float(ld), float(lm), float(ls), lh)
        lon = dms_to_decimal(float(od), float(om), float(os), oh)
        return lat, lon
    return None


def parse_any_location(text: str) -> Tuple[float, float]:
    link = parse_google_maps_link(text)
    if link is not None:
        return link
    return parse_latlon_text(text)


def google_maps_url_from_latlon(lat: float, lon: float, zoom: int = 12) -> str:
    return f"https://www.google.com/maps/@{lat:.6f},{lon:.6f},{zoom}z"


def google_maps_place_url(lat: float, lon: float) -> str:
    return f"https://www.google.com/maps?q={lat:.6f},{lon:.6f}"

# -----------------------------
# 2) NASA POWER — aylık klimatoloji (retry + cache)
# -----------------------------

POWER_PARAMS = [
    "T2M",         # 2m Temperature (°C)
    "T2M_MAX",     # 2m Tmax (°C)
    "T2M_MIN",     # 2m Tmin (°C)
    "PRECTOTCORR", # Precipitation (mm/day)
    "ALLSKY_SFC_SW_DWN", # All-sky shortwave (kWh/m2/day)
    "RH2M",        # 2m Relative Humidity (%)
    "WS2M",        # 2m Wind Speed (m/s)
]

MONTHS = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]


def _load_power_cache() -> Dict:
    if POWER_CACHE_FILE.exists():
        try:
            return json.loads(POWER_CACHE_FILE.read_text(encoding="utf-8"))
        except Exception:
            return {}
    return {}


def _save_power_cache(cache: Dict):
    try:
        POWER_CACHE_FILE.write_text(json.dumps(cache), encoding="utf-8")
    except Exception:
        pass


def _power_cache_key(lat: float, lon: float) -> str:
    # 0.01° (~1 km) çözünürlükte yuvarla, anahtar kısa kalsın
    return f"{round(lat, 2):.2f},{round(lon, 2):.2f}"


def fetch_power_climatology(lat: float, lon: float, retries: int = 3, backoff: float = 1.5) -> Dict[str, List[float]]:
    cache = _load_power_cache()
    key = _power_cache_key(lat, lon)
    if key in cache:
        return cache[key]

    params = ",".join(POWER_PARAMS)
    url = (
        "https://power.larc.nasa.gov/api/temporal/climatology/point"
        f"?parameters={params}&community=AG&longitude={lon}&latitude={lat}&format=JSON"
    )
    last_err = None
    for i in range(retries):
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            data = r.json()
            p = data["properties"]["parameter"]
            out = {}
            for keyp in POWER_PARAMS:
                out[keyp] = [p[keyp][m] for m in MONTHS]
            cache[key] = out
            _save_power_cache(cache)
            return out
        except Exception as e:
            last_err = e
            if i < retries - 1:
                time.sleep(backoff ** (i+1))
    # son denemede de hata:
    raise RuntimeError(f"NASA POWER isteği başarısız: {last_err}")

# -----------------------------
# 3) Aylık ET0 (Hargreaves) ve yardımcı iklim hesapları
# -----------------------------

GSC = 0.0820  # MJ m-2 min-1
PI = math.pi


def day_of_year_for_month_center(month: int) -> int:
    return [15, 46, 75, 105, 135, 162, 198, 228, 258, 288, 318, 344][month-1]


def extraterrestrial_radiation_MJm2day(lat_deg: float, month: int) -> float:
    phi = math.radians(lat_deg)
    J = day_of_year_for_month_center(month)
    dr = 1 + 0.033 * math.cos(2*PI*J/365)
    delta = 0.409 * math.sin(2*PI*J/365 - 1.39)
    ws = math.acos(-math.tan(phi) * math.tan(delta))
    Ra = (24*60/PI) * GSC * dr * (ws*math.sin(phi)*math.sin(delta) + math.cos(phi)*math.cos(delta)*math.sin(ws))
    return Ra  # MJ/m2/day


def eto_hargreaves_monthly(Tmean: float, Tmax: float, Tmin: float, Ra_MJm2day: float) -> float:
    dT = max(Tmax - Tmin, 0.1)
    return 0.0023 * (Tmean + 17.8) * math.sqrt(dT) * Ra_MJm2day


def days_in_month(m: int) -> int:
    return [31,28,31,30,31,30,31,31,30,31,30,31][m-1]

# -----------------------------
# 4) Ürün profilleri ve uygunluk
# -----------------------------

@dataclass
class CropProfile:
    name: str
    tbase: float
    season_months: List[int]
    gdd_target: int
    gdd_upper: int
    kc_mean: float
    alt_min: float = 0.0
    alt_max: float = 2000.0
    frost_tolerance: float = 0.0  # °C
    heat_thresh: float = 35.0     # °C


# Dahili varsayılan profiller (harici JSON yoksa kullanılır)
DEFAULT_CROPS: List[CropProfile] = [
    CropProfile("Buğday (kış)", tbase=5,  season_months=[10,11,12,1,2,3,4,5,6], gdd_target=1200, gdd_upper=2000, kc_mean=0.95, alt_max=1800, frost_tolerance=-10, heat_thresh=34),
    CropProfile("Mısır (dane)", tbase=10, season_months=[4,5,6,7,8,9],         gdd_target=1600, gdd_upper=2400, kc_mean=1.15, alt_max=1400, frost_tolerance=0,   heat_thresh=36),
    CropProfile("Ayçiçeği",     tbase=6,  season_months=[4,5,6,7,8,9],         gdd_target=1200, gdd_upper=2000, kc_mean=1.00, alt_max=1500, frost_tolerance=0,   heat_thresh=35),
    CropProfile("Domates",      tbase=10, season_months=[5,6,7,8,9],           gdd_target=1400, gdd_upper=2200, kc_mean=1.10, alt_max=1200, frost_tolerance=2,   heat_thresh=36),
    CropProfile("Biber",        tbase=10, season_months=[5,6,7,8,9],           gdd_target=1300, gdd_upper=2100, kc_mean=1.10, alt_max=1200, frost_tolerance=2,   heat_thresh=36),
    CropProfile("Bağ (üzüm)",   tbase=10, season_months=[4,5,6,7,8,9],         gdd_target=1600, gdd_upper=2500, kc_mean=0.90, alt_max=1300, frost_tolerance=-2,  heat_thresh=38),
    CropProfile("Zeytin",       tbase=12, season_months=[3,4,5,6,7,8,9,10,11], gdd_target=1800, gdd_upper=3000, kc_mean=0.70, alt_max=1000, frost_tolerance=-5,  heat_thresh=40),
]


def load_crop_profiles(path: Path) -> List[CropProfile]:
    if not path.exists():
        return DEFAULT_CROPS
    try:
        data = json.loads(path.read_text(encoding="utf-8"))
        profiles: List[CropProfile] = []
        for item in data:
            profiles.append(CropProfile(
                name=item["name"],
                tbase=float(item["tbase"]),
                season_months=[int(x) for x in item["season_months"]],
                gdd_target=int(item["gdd_target"]),
                gdd_upper=int(item.get("gdd_upper", int(item["gdd_target"]) * 2)),
                kc_mean=float(item.get("kc_mean", 1.0)),
                alt_min=float(item.get("alt_min", 0.0)),
                alt_max=float(item.get("alt_max", 2000.0)),
                frost_tolerance=float(item.get("frost_tolerance", 0.0)),
                heat_thresh=float(item.get("heat_thresh", 35.0)),
            ))
        return profiles
    except Exception as e:
        print(f"Uyarı: {path} okunamadı ({e}); dahili varsayılanlar kullanılacak.")
        return DEFAULT_CROPS


def trapezoid_score(x: float, a: float, b: float, c: float, d: float) -> float:
    if x <= a or x >= d:
        return 0.0
    if b <= x <= c:
        return 1.0
    if a < x < b:
        return (x - a) / (b - a)
    return (d - x) / (d - c)


def suitability_for_crop(crop: CropProfile, monthly: Dict[str, List[float]], lat: float, alt: Optional[float], irrigation_mode: str = IRRIGATION_MODE) -> Tuple[float, Dict[str, float]]:
    Tmean = monthly["T2M"]
    Tmax = monthly["T2M_MAX"]
    Tmin = monthly["T2M_MIN"]
    Pmm_day = monthly["PRECTOTCORR"]  # mm/day

    gdd = 0.0
    et0_sum = 0.0
    p_sum = 0.0

    for m in crop.season_months:
        idx = m - 1
        d = days_in_month(m)
        gdd += max(Tmean[idx] - crop.tbase, 0.0) * d
        Ra = extraterrestrial_radiation_MJm2day(lat, m)
        et0 = eto_hargreaves_monthly(Tmean[idx], Tmax[idx], Tmin[idx], Ra)
        et0_sum += et0 * d
        p_sum += max(Pmm_day[idx], 0.0) * d

    etc_sum = et0_sum * crop.kc_mean
    ratio = p_sum / max(etc_sum, 1e-6)
    water_score = trapezoid_score(ratio, 0.5, 0.8, 1.2, 1.8)

    gdd_score = trapezoid_score(gdd, 0.6*crop.gdd_target, 0.9*crop.gdd_target, 1.2*crop.gdd_target, crop.gdd_upper)

    season_tmax = max(Tmax[m-1] for m in crop.season_months)
    if season_tmax <= crop.heat_thresh:
        heat_score = 1.0
    else:
        heat_score = max(0.0, 1.0 - (season_tmax - crop.heat_thresh) / 10.0)

    season_tmin = min(Tmin[m-1] for m in crop.season_months)
    if season_tmin >= crop.frost_tolerance:
        frost_score = 1.0
    else:
        frost_score = max(0.0, 1.0 - (crop.frost_tolerance - season_tmin) / 5.0)

    # Rakım
    if alt is None:
        alt_score = 1.0
    else:
        if alt < crop.alt_min - 100 or alt > crop.alt_max + 200:
            alt_score = 0.0
        else:
            alt_score = min(
                1.0 if alt >= crop.alt_min + 100 else max(0.0, (alt - (crop.alt_min - 100)) / 200.0),
                1.0 if alt <= crop.alt_max - 200 else max(0.0, ((crop.alt_max + 200) - alt) / 400.0)
            )

    # Ağırlıklar — sulama moduna göre
    water_w = 0.35 if irrigation_mode == "rainfed" else 0.15
    base = 0.45 * gdd_score + water_w * water_score + 0.20 * heat_score
    final = base * frost_score * alt_score

    detail = {
        "gdd": gdd,
        "gdd_score": gdd_score,
        "p_sum_mm": p_sum,
        "et0_sum_mm": et0_sum,
        "etc_sum_mm": etc_sum,
        "water_ratio": ratio,
        "water_score": water_score,
        "season_tmax": season_tmax,
        "heat_score": heat_score,
        "season_tmin": season_tmin,
        "frost_score": frost_score,
        "altitude_m": alt if alt is not None else float("nan"),
        "alt_score": alt_score,
    }
    return final * 100.0, detail

# -----------------------------
# 5) Opsiyonel raster örnekleme
# -----------------------------

def sample_raster_value(path: Path, lon: float, lat: float) -> Optional[float]:
    if rasterio is None or not path.exists():
        return None
    try:
        with rasterio.open(path) as ds:
            row, col = ds.index(lon, lat)
            arr = ds.read(1)
            val = float(arr[row, col])
            if ds.nodata is not None and val == ds.nodata:
                return None
            return val
    except Exception:
        return None

# -----------------------------
# 6) Raporlama & çıktı dökümleri
# -----------------------------

MONTHS_TR = ["OCA","ŞUB","MAR","NİS","MAY","HAZ","TEM","AĞU","EYL","EKİ","KAS","ARA"]


def summarize_monthly(m: Dict[str, List[float]]) -> str:
    def fmt(x):
        return f"{x:5.1f}"
    lines = [
        "Ay   Tmin  Tmax  Tavg  Yağış(mm/g)  Radyasyon(kWh/m²/g)",
        "---  ----  ----  ----- -----------  -------------------",
    ]
    for i, mon in enumerate(MONTHS_TR, start=1):
        line = f"{mon} {fmt(m['T2M_MIN'][i-1])} {fmt(m['T2M_MAX'][i-1])} {fmt(m['T2M'][i-1])}     {m['PRECTOTCORR'][i-1]:6.2f}              {m['ALLSKY_SFC_SW_DWN'][i-1]:5.2f}"
        lines.append(line)
    return "\n".join(lines)


def write_results_json(path: Path, payload: Dict):
    try:
        path.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")
    except Exception as e:
        print(f"Uyarı: JSON yazılamadı: {e}")


def write_results_csv(path: Path, rows: List[Dict[str, object]]):
    try:
        if not rows:
            return
        cols = list(rows[0].keys())
        with path.open("w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=cols)
            w.writeheader()
            for r in rows:
                w.writerow(r)
    except Exception as e:
        print(f"Uyarı: CSV yazılamadı: {e}")

# -----------------------------
# 7) Ana akış
# -----------------------------

def main():
    raw = input("Konum girin (Google Maps linki YA DA 'lat,lon'): ").strip()
    lat, lon = parse_any_location(raw)

    maps_url = google_maps_url_from_latlon(lat, lon, zoom=12)
    maps_place_url = google_maps_place_url(lat, lon)

    print(f"\nKonum: {lat:.6f}, {lon:.6f}")
    print(f"Google Maps (kamera): {maps_url}")
    print(f"Google Maps (sorgu) : {maps_place_url}")

    print("\nVeriler çekiliyor (NASA POWER, 1991–2020 aylık ort.)...")
    monthly = fetch_power_climatology(lat, lon)

    elev = sample_raster_value(ELEV_CROPPED_PATH, lon, lat)
    light = sample_raster_value(LIGHT_CROPPED_PATH, lon, lat)

    print("\n— Aylık İklim Özeti —")
    print(summarize_monthly(monthly))

    if elev is not None:
        print(f"\nRakım (SRTM): {elev:.0f} m")
    if light is not None:
        print(f"Gece ışığı (VIIRS): {light:.3f}")

    crops = load_crop_profiles(CROP_CONFIG_PATH)
    results = []
    rows_csv: List[Dict[str, object]] = []

    for crop in crops:
        score, det = suitability_for_crop(crop, monthly, lat, elev, irrigation_mode=IRRIGATION_MODE)
        results.append((crop.name, score, det))
        rows_csv.append({
            "crop": crop.name,
            "score": round(score, 1),
            "gdd": round(det["gdd"], 0),
            "gdd_score": round(det["gdd_score"], 2),
            "water_ratio": round(det["water_ratio"], 2),
            "water_score": round(det["water_score"], 2),
            "season_tmax": round(det["season_tmax"], 1),
            "heat_score": round(det["heat_score"], 2),
            "season_tmin": round(det["season_tmin"], 1),
            "frost_score": round(det["frost_score"], 2),
            "altitude_m": None if math.isnan(det["altitude_m"]) else int(det["altitude_m"]),
            "alt_score": round(det["alt_score"], 2),
        })

    results.sort(key=lambda x: x[1], reverse=True)

    print("\n— Ürün Uygunluk Skoru (0–100) —")
    for name, score, det in results:
        reason = (
            f"GDD≈{det['gdd']:.0f} (skor {det['gdd_score']:.2f}); "
            f"Su indeksi P/ETc≈{det['water_ratio']:.2f} (skor {det['water_score']:.2f}); "
            f"Tmax_sezon≈{det['season_tmax']:.1f} (ısı skor {det['heat_score']:.2f}); "
            f"Tmin_sezon≈{det['season_tmin']:.1f} (don skor {det['frost_score']:.2f})"
        )
        if not math.isnan(det['altitude_m']):
            reason += f"; Rakım≈{det['altitude_m']:.0f} m (skor {det['alt_score']:.2f})"
        print(f"• {name:12s} → {score:5.1f} | {reason}")

    top3 = ", ".join([r[0] for r in results[:3]])
    print(f"\nSulama modu: {IRRIGATION_MODE}")
    print(f"Önerilen ilk 3: {top3}")

    # JSON & CSV dökümleri
    payload = {
        "location": {"lat": lat, "lon": lon, "maps_camera": maps_url, "maps_query": maps_place_url},
        "irrigation_mode": IRRIGATION_MODE,
        "monthly": monthly,
        "elevation_m": None if elev is None else float(elev),
        "night_light": None if light is None else float(light),
        "results_sorted": [
            {"crop": name, "score": round(score,1), "detail": det} for name, score, det in results
        ],
    }
    write_results_json(Path("results.json"), payload)
    write_results_csv(Path("results.csv"), rows_csv)


if __name__ == "__main__":
    main()


Konum girin (Google Maps linki YA DA 'lat,lon'):  39.96972876154957, 37.16724697349087



Konum: 39.969729, 37.167247
Google Maps (kamera): https://www.google.com/maps/@39.969729,37.167247,12z
Google Maps (sorgu) : https://www.google.com/maps?q=39.969729,37.167247

Veriler çekiliyor (NASA POWER, 1991–2020 aylık ort.)...

— Aylık İklim Özeti —
Ay   Tmin  Tmax  Tavg  Yağış(mm/g)  Radyasyon(kWh/m²/g)
---  ----  ----  ----- -----------  -------------------
OCA -23.4  10.5  -4.1       1.69               8.44
ŞUB -24.7  16.1  -2.0       1.31              12.50
MAR -16.9  21.7   2.1       1.63              14.62
NİS  -9.4  25.2   6.6       1.60              18.55
MAY  -3.3  30.1  11.5       1.98              22.36
HAZ  -0.1  32.8  15.8       1.23              26.92
TEM   4.0  37.4  18.6       0.45              28.19
AĞU   2.9  37.6  19.3       0.27              25.45
EYL   1.1  35.2  15.5       0.76              19.97
EKİ  -7.8  27.4  10.1       1.30              13.33
KAS -18.4  20.5   3.2       1.52               8.85
ARA -25.9  17.8  -1.8       1.38               6.51

Rakım (

In [22]:
import pandas as pd

# Yeni doğru yol
csv_path = r"C:..\notebooks\data\climate\merged_climate_data.csv"

# Sadece ilk satırı okuyarak sütun başlıklarını al
df = pd.read_csv(csv_path, nrows=0)

# Sütun isimlerini yazdır
print("📊 Dosyadaki sütun başlıkları:\n")
for col in df.columns:
    print("-", col)


📊 Dosyadaki sütun başlıkları:

- latitude
- longitude
- ALLSKY_SFC_PAR_TOT_grp1
- ALLSKY_SFC_SW_DWN_grp1
- CLOUD_AMT_grp1
- CLOUD_AMT_DAY_grp1
- CLOUD_AMT_NIGHT_grp1
- CLRSKY_DAYS_grp1
- CLRSKY_SFC_SW_DWN_grp1
- QV2M_grp2
- RH2M_grp2
- T2M_grp2
- T2MDEW_grp2
- T2MWET_grp2
- T2M_MAX_grp2
- T2M_MIN_grp2
- T2M_RANGE_grp2
- TQV_grp2
- PS_grp3
- SLP_grp3
- WD2M_grp3
- WS2M_grp3
- WS2M_MAX_grp3
- PRECTOTCORR_grp4
- TO3_grp4
- TS_grp4


In [26]:
import pandas as pd
import numpy as np

# CSV yolu
csv_path = r"C:..\notebooks\data\climate\merged_climate_data.csv"

# İncelenecek konum
lat = 41.239705
lon = 41.915686

# CSV'yi oku
df = pd.read_csv(csv_path)

# Haversine fonksiyonu (iki nokta arası mesafe, km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Tüm satırlar için mesafe hesapla
df["distance_km"] = haversine(lat, lon, df["latitude"], df["longitude"])

# En yakın satırı bul
closest = df.loc[df["distance_km"].idxmin()]

# Sonucu göster
print(f"🎯 En yakın nokta ({closest['latitude']:.4f}, {closest['longitude']:.4f}) - {closest['distance_km']:.2f} km uzaklıkta\n")

# İlk 15 sütunu sade şekilde göster
print(closest.head(15))


🎯 En yakın nokta (41.0000, 42.0000) - 27.57 km uzaklıkta

latitude                   41.00
longitude                  42.00
ALLSKY_SFC_PAR_TOT_grp1     0.88
ALLSKY_SFC_SW_DWN_grp1      1.94
CLOUD_AMT_grp1             87.13
CLOUD_AMT_DAY_grp1         79.47
CLOUD_AMT_NIGHT_grp1       92.17
CLRSKY_DAYS_grp1            0.00
CLRSKY_SFC_SW_DWN_grp1      2.91
QV2M_grp2                   2.46
RH2M_grp2                  91.05
T2M_grp2                   -7.08
T2MDEW_grp2                -8.68
T2MWET_grp2                -7.88
T2M_MAX_grp2                8.80
Name: 3913, dtype: float64


In [28]:
import pandas as pd
import numpy as np
from pathlib import Path

# 🗂️ Verilerin bulunduğu dosya yolları
csv_path = Path(r"C:..\notebooks\data\climate\merged_climate_data.csv")
elev_path = Path(r"C:..\data\processed\srtm_turkiye_cropped.tif")
light_path = Path(r"C:..\data\processed\viirs_light_2024_turkey.tif")

# 📍 İncelenecek konum
lat = 41.239705
lon = 41.915686

# -------------------
# Yardımcı fonksiyonlar
# -------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def sample_raster(path, lon, lat):
    try:
        import rasterio
        with rasterio.open(path) as ds:
            row, col = ds.index(lon, lat)
            val = ds.read(1)[row, col]
            if ds.nodata is not None and val == ds.nodata:
                return None
            return float(val)
    except Exception:
        return None

# -------------------
# 1️⃣ İklim verisini CSV'den al
# -------------------
df = pd.read_csv(csv_path)
df["distance_km"] = haversine(lat, lon, df["latitude"], df["longitude"])
closest = df.loc[df["distance_km"].idxmin()].copy()
dist_km = float(closest["distance_km"])

# -------------------
# 2️⃣ Opsiyonel: Rakım ve Işık verisi
# -------------------
elevation = sample_raster(elev_path, lon, lat)
light = sample_raster(light_path, lon, lat)

# -------------------
# 3️⃣ Hepsini bir araya getir
# -------------------
data_dict = closest.to_dict()
data_dict["distance_km"] = dist_km
if elevation is not None:
    data_dict["Elevation_m"] = round(elevation, 2)
if light is not None:
    data_dict["Night_Light"] = round(light, 3)

# -------------------
# 4️⃣ Sonuçları göster
# -------------------
print(f"📍 Girdi Konum: ({lat:.5f}, {lon:.5f})")
print(f"🎯 En yakın nokta: ({data_dict['latitude']:.3f}, {data_dict['longitude']:.3f}) ≈ {dist_km:.2f} km uzaklıkta\n")

# DataFrame olarak yazdır
result_df = pd.DataFrame(list(data_dict.items()), columns=["Değişken", "Değer"])
display(result_df)


📍 Girdi Konum: (41.23971, 41.91569)
🎯 En yakın nokta: (41.000, 42.000) ≈ 27.57 km uzaklıkta



,Değişken,Değer
0,latitude,41.00000
1,longitude,42.00000
2,ALLSKY_SFC_PAR_TOT_grp1,0.88000
3,ALLSKY_SFC_SW_DWN_grp1,1.94000
4,CLOUD_AMT_grp1,87.13000
5,CLOUD_AMT_DAY_grp1,79.47000
6,CLOUD_AMT_NIGHT_grp1,92.17000
7,CLRSKY_DAYS_grp1,0.00000
8,CLRSKY_SFC_SW_DWN_grp1,2.91000
9,QV2M_grp2,2.46000


In [30]:
import pandas as pd

# 🔹 Mevcut veriler (örnek senin çıktın)
data = {
    "latitude": 41.00000,
    "longitude": 42.00000,
    "ALLSKY_SFC_PAR_TOT_grp1": 0.88,
    "ALLSKY_SFC_SW_DWN_grp1": 1.94,
    "CLOUD_AMT_grp1": 87.13,
    "CLOUD_AMT_DAY_grp1": 79.47,
    "CLOUD_AMT_NIGHT_grp1": 92.17,
    "CLRSKY_DAYS_grp1": 0.00,
    "CLRSKY_SFC_SW_DWN_grp1": 2.91,
    "QV2M_grp2": 2.46,
    "RH2M_grp2": 91.05,
    "T2M_grp2": -7.08,
    "T2MDEW_grp2": -8.68,
    "T2MWET_grp2": -7.88,
    "T2M_MAX_grp2": 8.80,
    "T2M_MIN_grp2": -25.31,
    "T2M_RANGE_grp2": 10.08,
    "TQV_grp2": 5.55,
    "PS_grp3": 83.25,
    "SLP_grp3": 101.89,
    "WD2M_grp3": 176.80,
    "WS2M_grp3": 1.43,
    "WS2M_MAX_grp3": 8.83,
    "PRECTOTCORR_grp4": 3.39,
    "TO3_grp4": 332.61,
    "TS_grp4": -10.00,
    "Elevation_m": 1469.00,
    "Night_Light": 0.00,
}

# 🔸 Açıklama sözlüğü (NASA POWER + ek veriler)
explanations = {
    "latitude": "Enlem (°)",
    "longitude": "Boylam (°)",
    "ALLSKY_SFC_PAR_TOT_grp1": "Fotosentetik aktif radyasyon (PAR) – bitki büyümesi için gerekli ışık enerjisi [MJ/m²/gün]",
    "ALLSKY_SFC_SW_DWN_grp1": "Toplam kısa dalga güneş radyasyonu – yüzeye ulaşan enerji [kWh/m²/gün]",
    "CLOUD_AMT_grp1": "Ortalama bulutluluk oranı [%]",
    "CLOUD_AMT_DAY_grp1": "Gündüz bulutluluk oranı [%]",
    "CLOUD_AMT_NIGHT_grp1": "Gece bulutluluk oranı [%]",
    "CLRSKY_DAYS_grp1": "Aylık açık gün sayısı [gün/ay]",
    "CLRSKY_SFC_SW_DWN_grp1": "Bulutsuz koşullardaki kısa dalga radyasyon [kWh/m²/gün]",
    "QV2M_grp2": "2 metrede özgül nem (su buharı miktarı) [g/kg]",
    "RH2M_grp2": "2 metrede bağıl nem [%]",
    "T2M_grp2": "2 metrede ortalama hava sıcaklığı [°C]",
    "T2MDEW_grp2": "Çiy noktası sıcaklığı [°C]",
    "T2MWET_grp2": "Yaş termometre sıcaklığı [°C]",
    "T2M_MAX_grp2": "Maksimum günlük sıcaklık [°C]",
    "T2M_MIN_grp2": "Minimum günlük sıcaklık [°C]",
    "T2M_RANGE_grp2": "Günlük sıcaklık farkı (maks-min) [°C]",
    "TQV_grp2": "Toplam kolondaki su buharı miktarı [kg/m²]",
    "PS_grp3": "Yüzey basıncı [kPa]",
    "SLP_grp3": "Deniz seviyesine indirgenmiş basınç [kPa]",
    "WD2M_grp3": "2 metrede ortalama rüzgar yönü [°]",
    "WS2M_grp3": "2 metrede ortalama rüzgar hızı [m/s]",
    "WS2M_MAX_grp3": "2 metrede maksimum rüzgar hızı [m/s]",
    "PRECTOTCORR_grp4": "Toplam yağış (düzeltilmiş) [mm/gün]",
    "TO3_grp4": "Toplam ozon kolonu [Dobson Unit]",
    "TS_grp4": "Yüzey sıcaklığı [°C]",
    "Elevation_m": "Rakım [metre]",
    "Night_Light": "Gece ışığı (yapay aydınlatma yoğunluğu, VIIRS verisi)",
}

# 🔹 Tablo oluştur
rows = []
for key, value in data.items():
    desc = explanations.get(key, "(Açıklama bulunamadı)")
    rows.append([key, value, desc])

df = pd.DataFrame(rows, columns=["Veri Başlığı", "Değer", "Açıklama"])
display(df)


,Veri Başlığı,Değer,Açıklama
0,latitude,41.00,Enlem (°)
1,longitude,42.00,Boylam (°)
2,ALLSKY_SFC_PAR_TOT_grp1,0.88,Fotosentetik aktif radyasyon (PAR) – bitki büy...
3,ALLSKY_SFC_SW_DWN_grp1,1.94,Toplam kısa dalga güneş radyasyonu – yüzeye ul...
4,CLOUD_AMT_grp1,87.13,Ortalama bulutluluk oranı [%]
5,CLOUD_AMT_DAY_grp1,79.47,Gündüz bulutluluk oranı [%]
6,CLOUD_AMT_NIGHT_grp1,92.17,Gece bulutluluk oranı [%]
7,CLRSKY_DAYS_grp1,0.00,Aylık açık gün sayısı [gün/ay]
8,CLRSKY_SFC_SW_DWN_grp1,2.91,Bulutsuz koşullardaki kısa dalga radyasyon [kW...
9,QV2M_grp2,2.46,2 metrede özgül nem (su buharı miktarı) [g/kg]
